# Short demo of using `ipython_memory_usage` to diagnose numpy and Pandas RAM usage

Author Ian uses this tool in his Higher Performance Python training (https://ianozsvald.com/training/) and it is mentioned in his High Performance Python (2nd ed, O'Reilly) technical book.

We can use it to understand how much RAM we're currently using and which of several alternate ways to solve a problem in complex tools might be the most RAM efficient solutions.

* `total RAM usage` is the current RAM usage at the end of that cell's execution
* `used` shows the difference between the _last_ `total RAM usage` and this one
* `peaked` shows any during-execution peak _above_ the resulting `total RAM usage` (i.e. hidden RAM usage that might catch you out)

In [1]:
import ipython_memory_usage
help(ipython_memory_usage) # or ipython_memory_usage?

Help on package ipython_memory_usage:

NAME
    ipython_memory_usage - Profile mem usage envelope of IPython commands and report interactively

DESCRIPTION
    Use 
    In[] import ipython_memory_usage 
    In[] %ipython_memory_usage_start # invoke magic-based tracking and
    # %ipython_memory_usage_stop to disable

PACKAGE CONTENTS
    ipython_memory_usage
    ipython_memory_usage_perf
    perf_process

SUBMODULES
    imu

FUNCTIONS
    ipython_memory_usage_start(line, cell=None)
    
    ipython_memory_usage_stop(line, cell=None)

FILE
    /home/ian/workspace/personal_projects/ipython_memory_usage_base/ipython_memory_usage/ipython_memory_usage_dev/ipython_memory_usage/src/ipython_memory_usage/__init__.py




In [2]:
%ipython_memory_usage_start

'memory profile enabled'

In [2] used 0.1562 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 48.67 MiB


# Importing packages uses some RAM

In [3]:
import numpy as np # note that importing a package will increase total RAM usage a little

In [3] used 12.0039 MiB RAM in 0.24s, peaked 0.00 MiB above current, total RAM usage 60.68 MiB


In [4]:
import pandas as pd # note that importing Pandas uses more RAM than importing numpy

In [4] used 24.3984 MiB RAM in 0.35s, peaked 0.00 MiB above current, total RAM usage 85.07 MiB


In [5]:
import string

In [5] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 85.07 MiB


# Making a large array uses a predictable amount of RAM

In [6]:
# if we make a big array - 100M items * 8 byte floats, this cell
# uses circa 800MB (often 760 MiB - note mibi-bytes as used in the underlying memory_profiler tool)
# The total RAM usage grows by roughly this amount
arr = np.ones(100_000_000) 

In [6] used 763.3750 MiB RAM in 0.25s, peaked 0.00 MiB above current, total RAM usage 848.45 MiB


In [7]:
# deleting arr reduces RAM usage by roughly the expected amount and
# total RAM usage should drop back down
del arr

In [7] used -762.9297 MiB RAM in 0.11s, peaked 762.93 MiB above current, total RAM usage 85.52 MiB


In [8]:
# if we make it again, RAM usage goes up again
arr = np.ones(100_000_000) 

In [8] used 763.0039 MiB RAM in 0.24s, peaked 0.00 MiB above current, total RAM usage 848.52 MiB


In [9]:
del arr

In [9] used -762.6836 MiB RAM in 0.11s, peaked 762.68 MiB above current, total RAM usage 85.84 MiB


# Making a big random array takes RAM + time

In [10]:
# creating random items takes some time, after "used ... RAM" note "3s" or so for several seconds
arr = np.random.normal(size=100_000_000)
print(arr[:5], arr.dtype)

[-0.14678642  0.14768542 -1.1194974  -0.83116093 -0.30498602] float64
In [10] used 763.0117 MiB RAM in 3.26s, peaked 0.00 MiB above current, total RAM usage 848.85 MiB


# Intermediate calculations can cost additional temporary RAM 

**NOTE** this section may work different if you're on Windows (if so - please report back to Ian by raising a bug and noting the difference.

On some platforms, e.g. Linux as used here, temporary intermediates can be reused in-place reducing the overall memory allocation: https://docs.scipy.org/doc/numpy-1.13.0/release.html#highlights

In [11]:
pass

In [11] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 848.85 MiB


In [12]:
# arr*2 and arr*3 both have to be stored somewhere before the division can occur
# so two more circa 762MiB arrays are made temporarily, this is reported
# as "peaked 762MiB above current"
# before they can be discard. arr_result references the final result
# so overall we add 762MiB to the process
# we only add 762MiB, not 762MiB*2, as on Linux we can intelligently reuse
# one of the temporaries (else we'd peak at 762*2 MiB)

# we report "used 762...MiB" as the final arr_result adds this to the process
# so overall we're now _at_ 1.6GB but we actually peaked at 1.6+0.7 == 2.3GB 
# whilst this cell executed
# if your code crashes with an out of memory exception, it could be caused
# by a situation like this
arr_result = (arr * 2) / (arr * 3)

In [12] used 763.1992 MiB RAM in 0.58s, peaked 762.68 MiB above current, total RAM usage 1612.05 MiB


In [13]:
del arr

In [13] used -762.9219 MiB RAM in 0.10s, peaked 762.92 MiB above current, total RAM usage 849.13 MiB


In [14]:
del arr_result

In [14] used -762.9180 MiB RAM in 0.10s, peaked 762.92 MiB above current, total RAM usage 86.21 MiB


# Pandas DataFrames can be costly on RAM

## Example with deleting columns

Props to Jamie Brunning for this example

In [15]:
pass

In [15] used 0.1406 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 86.35 MiB


In [16]:
arr_several_cols = np.random.normal(size=(100_000_000, 4))

In [16] used 3051.7812 MiB RAM in 12.77s, peaked 0.00 MiB above current, total RAM usage 3138.13 MiB


In [17]:
arr_several_cols.shape

(100000000, 4)

In [17] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 3138.13 MiB


In [18]:
f"Cost per column {int(arr_several_cols.data.nbytes / arr_several_cols.shape[1]):,} bytes"

'Cost per column 800,000,000 bytes'

In [18] used 0.0156 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 3138.15 MiB


In [19]:
# The DataFrame in this case is a thin wrapper over the numpy array
# and costs little extra RAM
df = pd.DataFrame(arr_several_cols, columns=list(string.ascii_lowercase)[:arr_several_cols.shape[1]])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a       float64
 1   b       float64
 2   c       float64
 3   d       float64
dtypes: float64(4)
memory usage: 3.0 GB
In [19] used 1.2383 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 3139.39 MiB


In [20]:
# use Jupyter's xdel to remove all references of our expensive array, just in case
# (but not in this case) it is also referred to in an Out[] history item
%xdel arr_several_cols

In [20] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 3139.39 MiB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   a       float64
 1   b       float64
 2   c       float64
 3   d       float64
dtypes: float64(4)
memory usage: 3.0 GB
In [21] used 0.0000 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 3139.39 MiB


In [22]:
# using del is surprisingly expensive
# total RAM usage goes up by circa 1.5GB-2GB (>2x the cost of 1 column) 
# DOES ANYONE KNOW WHAT'S HAPPENING BEHIND THE SCENES HERE?
# THE NEXT 2 CELLS SHOW IT ISN'T BEING QUICKLY GARBAGE COLLECTED
# note also that using del seems to take more seconds than using df.drop (a few cells below)
# possibly internally there's now (somehow) a 4-column original array _and_ a
# 3 column resulting array (in the BlockManager?) costing 7-columns (i.e. circa 800MB*7 == circa 5.6GB)
del df['a']

In [22] used 2289.1992 MiB RAM in 1.04s, peaked 0.00 MiB above current, total RAM usage 5428.59 MiB


In [23]:
# we get no benefit by forcing a collection
import gc
gc.collect()

60

In [23] used 0.0430 MiB RAM in 0.14s, peaked 0.00 MiB above current, total RAM usage 5428.63 MiB


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   b       float64
 1   c       float64
 2   d       float64
dtypes: float64(3)
memory usage: 2.2 GB
In [24] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 5428.63 MiB


In [25]:
pass

In [25] used 0.0352 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 5428.66 MiB


In [26]:
# using drop with inplace=False (the default) returns a copied DataFrame, if you don't use
# this then maybe you end up with multiple DataFrames consuming RAM in a confusing fashion
# e.g. you might have done `df2 = df.drop...` and then you've got the unmodified original
# plus the modified df2 in the local namespace
# We see total RAM usage drop by circa 800MB, the cost of 1 column, plus a lot more...
# which is a mystery to me!
# maybe the usage of drop forces a flush on any internal caching in pandas?
df = df.drop(columns=['b'])

In [26] used -3814.5781 MiB RAM in 0.52s, peaked 3814.58 MiB above current, total RAM usage 1614.09 MiB


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   c       float64
 1   d       float64
dtypes: float64(2)
memory usage: 1.5 GB
In [27] used -0.0156 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 1614.07 MiB


In [28]:
# dropping in-place is probably more sensible, we recover another circa 800MB
df.drop(columns=['c'], inplace=True)

In [28] used -762.9336 MiB RAM in 0.32s, peaked 0.00 MiB above current, total RAM usage 851.14 MiB


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000000 entries, 0 to 99999999
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   d       float64
dtypes: float64(1)
memory usage: 762.9 MB
In [29] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 851.14 MiB


In [30]:
pass

In [30] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 851.14 MiB


In [31]:
# now we get back to where we were before we made the DataFrame and the array
df.drop(columns=['d'], inplace=True)

In [31] used -762.9258 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 88.21 MiB


# Diagnostics

`%xdel my_df` will delete all references of `my_df` from the namespace including those in the Out[] history buffer, this does more cleaning than just using `del my_df`.

`%reset` will reset all variables and imported modules, it is like starting a new kernel.

In [32]:
# %whos shows what's in the local namespace
%whos

Variable               Type         Data/Info
---------------------------------------------
df                     DataFrame    Empty DataFrame\nColumns:<...>0000000 rows x 0 columns]
gc                     module       <module 'gc' (built-in)>
ipython_memory_usage   module       <module 'ipython_memory_u<...>emory_usage/__init__.py'>
np                     module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd                     module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
string                 module       <module 'string' from '/h<...>lib/python3.9/string.py'>
In [32] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 88.21 MiB


In [33]:
# we can use %xdel to safely remove all references including those that might be (but not in this case)
# in the Out[] history buffer
%xdel df

In [33] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 88.21 MiB


In [34]:
%ipython_memory_usage_stop

'memory profile disabled'